In [ ]:
#importing the training and validation data
from google.colab import drive
drive.mount('/content/data')

Mounted at /content/data


In [ ]:
#import libraries
import yaml
import numpy as np
import os
import cv2
import random

#Global variables
data_yaml = [] #This is data that is stored in the yaml file

#load data from yaml files
with open('/content/data/My Drive/AI_Project/data.yaml', 'r') as file:
  data = yaml.safe_load(file)

#print(data) #remove this later

#loading training images and labels
train_images = data['train_images']
train_labels = data['train_labels']

#loading validate images and labels
valid_images = data['val_images']
valid_labels = data['val_labels']

#loading test images and labels
test_images = data['test_images']
test_labels = data['test_labels']

#Loading images and labels into arrays
def load_X_Y_values(image_path, label_path):
  images = []
  labels = []
  max_label_length = 10

  #open image folder and for each image
  for filename in os.listdir(image_path):
    img_name = filename.split('.')[0] #note the image name eg img_name.png
    img_path = os.path.join(image_path, filename)

    #read the original image
    image = cv2.imread(img_path)

    #resize the original image
    image = cv2.resize(image, (224, 224)) #224 seems to be the standard size

    images.append(image)

    # Apply rotation and translation augmentation
    # Randomly rotate the image by -15 to +15 degrees
    angle = random.randint(-15, 15)
    rows, cols, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

    # Randomly translate the image by -10 to +10 pixels horizontally and vertically
    tx = random.randint(-10, 10)
    ty = random.randint(-10, 10)
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, (cols, rows))

    # Append rotated and translated images
    images.append(rotated_image)
    images.append(translated_image)

    #open the label folder
    label_file_path = os.path.join(label_path, img_name + '.txt')
    if os.path.exists(label_file_path):
      label_vector = np.zeros(max_label_length)  # Initialize label vector
      with open(label_file_path, 'r') as label_file:
        # Extract class indices from the lines
        indexes = [int(line.split()[0]) for line in label_file]
        # Mark indexes to corresponding lael
        label_vector[indexes] = 1

        # Append for original image, rotated image, and translated image
        labels.append(label_vector)
        labels.append(label_vector)
        labels.append(label_vector)

    else: #append empty array as label if file doesnt exist
       labels.extend([np.zeros(max_label_length)] * 3)


  return np.array(images), np.array(labels)

#load the training images and labels
train_image, train_label = load_X_Y_values(train_images, train_labels)

#load the testing images and labels
test_image, test_label = load_X_Y_values(test_images, test_labels)

#load the validation images and labels
valid_image, valid_label = load_X_Y_values(valid_images, valid_labels)

#shape of images and labels
print('Image ', len(train_image)) #Image  3036
print('Tags ', len(train_label)) #Tags  3036
#print(train_label)

Image  3036
Tags  3036


In [ ]:
import tensorflow as tf
import random


# Define Keras model creation function
def create_model(optimizer='adam', activation='relu', kernel_initializer='he_uniform', neurons=256):
    input_shape = (224, 224, 3)
    output_shape = 10

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.01), kernel_initializer='he_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Conv2D(64, (3, 3), activation=activation, kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Conv2D(128, (3, 3), activation=activation, kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.4),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(neurons, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),

        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(neurons, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),

        tf.keras.layers.Dense(output_shape, activation='sigmoid')
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


# Define custom scoring function
def custom_scoring(estimator, X, y):
    _, accuracy = estimator.evaluate(X, y)
    return accuracy

# Define hyperparameters grid for grid search
param_grid = {
    'batch_size': [32, 64],
    'epochs': [10, 20],
    'optimizer': ['adam', 'sgd'],
    'activation': ['relu', 'tanh'],
    'neurons': [8, 12, 16]
}

best_model = None
best_score = 0

for batch_size in param_grid['batch_size']:
    for epochs in param_grid['epochs']:
        for optimizer in param_grid['optimizer']:
            for activation in param_grid['activation']:
                for neurons in param_grid['neurons']:
                    model = create_model(optimizer=optimizer, activation=activation, neurons=neurons)
                    history = model.fit(train_image, train_label, epochs=epochs, batch_size=batch_size, validation_data=(valid_image, valid_label))
                    val_accuracy = history.history['val_accuracy'][-1]

                    if val_accuracy > best_score:
                        best_score = val_accuracy
                        best_model = model

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(test_image, test_label)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Epoch 1/10
95/95 [==============================] - 842s 9s/step - loss: 7.6690 - accuracy: 0.0122 - val_loss: 4.5638 - val_accuracy: 0.0043
Epoch 2/10
95/95 [==============================] - 809s 9s/step - loss: 2.7979 - accuracy: 0.0082 - val_loss: 2.4753 - val_accuracy: 0.0087
Epoch 3/10
95/95 [==============================] - 765s 8s/step - loss: 2.0153 - accuracy: 0.1028 - val_loss: 1.7581 - val_accuracy: 0.0580
Epoch 4/10
95/95 [==============================] - 759s 8s/step - loss: 1.3149 - accuracy: 0.3617 - val_loss: 2.5324 - val_accuracy: 0.0116
Epoch 5/10
95/95 [==============================] - 755s 8s/step - loss: 1.0593 - accuracy: 0.3603 - val_loss: 1.1802 - val_accuracy: 0.2710
Epoch 6/10
95/95 [==============================] - 754s 8s/step - loss: 0.8437 - accuracy: 0.3673 - val_loss: 0.7490 - val_accuracy: 0.3174
Epoch 7/10
95/95 [==============================] - 760s 8s/step - loss: 0.6898 - accuracy: 0.3663 - val_loss: 0.6279 - val_accuracy: 0.3652
Epoch 8/10
95

In [ ]:
import tensorflow as tf

#Increasing the training data set size
#Defining image generator
training_data_augmentation = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.25,
    zoom_range = 0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.5, 1.5],
    channel_shift_range=50,
    rescale=1./255
)

#using data generator to increase data size
train_generator = training_data_augmentation.flow(train_image, train_label, batch_size = 32)

print("Training Data Generated")
print("Length ", len(train_generator))


Training Data Generated
Length  95


In [ ]:
import tensorflow as tf

#Customizing the Learning Rate with Adam Optimizer
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

#The CNN model
input_shape = (224, 224, 3) #height, width, rgb count
output_shape = 10

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, kernel_regularizer=tf.keras.regularizers.l2(0.01), kernel_initializer='he_uniform'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.03), kernel_initializer='he_uniform'),


    tf.keras.layers.Dense(output_shape, activation='sigmoid')  # Sigmoid activation
])

#compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#train the model
model.fit(train_generator, epochs=15, validation_data=(valid_image, valid_label), callbacks=[early_stopping])

#evaluate the model
test_loss, test_accuracy = model.evaluate(test_image, test_label)

#print the accuracy --> correctly predicted labels
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/15
63/95 [==================>...........] - ETA: 7:41 - loss: 105.1162 - accuracy: 0.2470

KeyboardInterrupt: 